In [1]:
# import libraries
import os
import numpy as np
from tqdm import tqdm

from ifum_stitch import Stitch
from ifum_maskopt import Mask
from ifum_rectify import Rectify
from ifum_calibrate import Calibrate
from ifum_stack import Stack

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# directory containing unprocessed files

directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240212\\"
# directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240210\\"


# all files included in a single stack, repeat where necessary
# only include string in file that includes all files from single exposure

# data_filenames = ["1156","1157","1158","1162","1163","1164","1168","1169","1170"]
# arc_filenames = ["1161","1161","1161","1167","1167","1167","1172","1172","1172"]
# flat_filenames = ["1160","1160","1160","1166","1166","1166","1171","1171","1171"]
# data_filenames = ["0721","0722","0723","0727","0728","0729","0736","0737","0738"]
# arc_filenames = ["0725","0725","0725","0733","0733","0733","0740","0740","0740"]
# flat_filenames = ["0724","0724","0724","0734","0734","0734","0739","0739","0739"]
data_filenames = ["1177","1178"]
arc_filenames = ["1180","1180"]
flat_filenames = ["1179","1179"]
# data_filenames = ["0746","0747","0748"]
# arc_filenames = ["0745","0745","0745"]
# flat_filenames = ["0744","0744","0744"]
# data_filenames = ["0756","0757","0758"]
# arc_filenames = ["0755","0755","0755"]
# flat_filenames = ["0754","0754","0754"]
# data_filenames = ["1148","1149","1150","1151","1152"]
# arc_filenames = ["1161","1161","1161","1161","1161"]
# flat_filenames = ["1160","1160","1160","1160","1160"]

# mask???
# mode LR,STD,HR
mode = "HR"

# far red, blue, ...
wavelength = "blue" #"far red"

# bad masks (on scale 1-x)
# bad_blues = [23]
# bad_reds = []
bad_blues = [271,298,351]
bad_reds = [189,216,243]

# stars to use in WCS (list RA,Dec)
# all stars should be present in at least some dithers

# wcs_stars = [[74.8322, -58.6579],
#              [74.8279, -58.6548],
#              [74.8314, -58.6526],
#              [74.8254, -58.6572],
#              [74.8303, -58.6543]]
wcs_stars = [[74.8322, -58.6579],
             [74.8305, -58.6603],
             [74.8308, -58.6587],
             [74.8254, -58.6572],
             [74.8237, -58.6594]]

bin_to_2x1 = True

# things to add...
# plot = False (generic plots)
# debug_mode = False (extra print statements, plots, etc.)
# overwrite = False (if file exists, assume already processed correctly)
# sub_steps = False (show smaller TQDM for each mask where relevant)

In [3]:
# assert statements!!!
bad_masks = [np.array(bad_blues)-1,np.array(bad_reds)-1]
wcs_stars = np.array(wcs_stars)
if mode == "STD":
    total_masks = 552
    mask_groups = 12
    hex_dims = (23,24)
elif mode == "HR":
    total_masks = 864
    mask_groups = 16
    hex_dims = (27,32)
else:
    print("invalid mode")

if wavelength == "far red":
    bins = np.arange(7000,10000,1)
elif wavelength == "blue":
    bins = np.arange(4000,6100,1)
else:
    print("invalid wavelength")

<h1><strong><span style="color:purple">STITCH</h1>

<span style="color:orange"><strong>creates 2 files, one for each detector, for both the data and arc files. also produces a cosmic ray mask for the data.

In [4]:
# out directory where all files are stored
if not os.path.exists("out"):
    os.makedirs("out")

# stitch and create file
for file in tqdm(data_filenames+arc_filenames+flat_filenames):
    file_to_stitch = Stitch(directory,file,None,None,None,None,None)
    file_to_stitch.load_files()
    file_to_stitch.save_file(bin_to_2x1)
print("stitched files saved")

100%|██████████| 6/6 [00:09<00:00,  1.57s/it]

stitched files saved


In [5]:
# use flat and median filter to get rid of internal bias
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_bias = Stitch(directory,None,None,"b",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
    file_for_bias = Stitch(directory,None,None,"r",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
print("internal bias solved")

100%|██████████| 2/2 [00:07<00:00,  3.81s/it]

internal bias solved


In [6]:
# create cosmic ray masks
for datafilename in tqdm(data_filenames):
    file_for_cmray = Stitch(directory,None,None,"b",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
    file_for_cmray = Stitch(directory,None,None,"r",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
print("cosmic ray masks created")

100%|██████████| 2/2 [00:40<00:00, 20.31s/it]

cosmic ray masks created


<h1><strong><span style="color:purple">OPTIMIZE MASK</h1>

<span style="color:orange"><strong>use current mask for single gaussian fits to get better mask guess  

In [7]:
# first guess; complex guess
for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

  0%|          | 0/1 [00:00<?, ?it/s]

saving...
saved
saving...
saved


100%|██████████| 1/1 [13:13<00:00, 793.22s/it]


In [4]:
# change the degrees until fully confident with all the fits!
center_deg = 5
sigma_deg = 3

for flatfilename in np.unique(flat_filenames):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)

In [5]:
# TODO: calculate sig_mult based on mask non-overlap
sig_mult = 1.5

for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)

100%|██████████| 1/1 [02:17<00:00, 137.60s/it]


In [4]:
# optimize arc files
expected_peaks = 25#15
sig_mult = 1.5
optimize = True # depends if not enough lines

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks,optimize=optimize)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks,optimize=optimize)

100%|██████████| 1/1 [17:30<00:00, 1050.00s/it]


In [5]:
# optimize data files
expected_peaks = 10#25
sig_mult = 1.5
optimize = False

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks,optimize=optimize)
    file_for_mask.get_rots(arcfilename,datafilename,optimize=optimize)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks,optimize=optimize)
    file_for_mask.get_rots(arcfilename,datafilename,optimize=optimize)

100%|██████████| 2/2 [09:08<00:00, 274.24s/it]


In [6]:
# create masks for arc!
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)

100%|██████████| 1/1 [04:15<00:00, 255.81s/it]


In [7]:
# create masks for data
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename,copy=arcfilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename,copy=arcfilename)

100%|██████████| 2/2 [06:55<00:00, 207.69s/it]


In [4]:
# TEMP VIZ
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask._viz_(arcfilename,"b",3)

  0%|          | 0/2 [00:00<?, ?it/s]

line 2
(429, 12, 7)
line 10
[-0.24643355  0.35356368  1.08151821  0.60920223  0.02138887  0.15116475
  0.00852855]
7
8
9
10
11


 50%|█████     | 1/2 [01:08<01:08, 68.98s/it]

line 2
(429, 12, 7)
line 10
[-0.24643355  0.35356368  1.08151821  0.60920223  0.02138887  0.15116475
  0.00852855]
7
8
9
10
11


100%|██████████| 2/2 [01:14<00:00, 37.11s/it]


<h1><strong><span style="color:purple">RECTIFY</h1>

<span style="color:orange"><strong>rectify + calibrate

In [ ]:
# TEMP
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect._viz_wl()

  0%|          | 0/2 [00:00<?, ?it/s]

activated


In [4]:
# usualy it's a good idea to fix sparse if you expect a good density of peaks throughout
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("data")
    
    file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("data")

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",fix_sparse=True)

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",fix_sparse=True)

100%|██████████| 1/1 [04:26<00:00, 266.47s/it]


In [5]:
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

    file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

  0%|          | 0/2 [00:00<?, ?it/s]

BAD emission line: 1616.918175810298
BAD emission line: 1645.3339554077486
BAD emission line: 1687.1855584186264
BAD emission line: 1734.694587058697
GOOD emission line: 1908.7104835600865
GOOD emission line: 1930.6988689137763
BAD emission line: 1974.543407982069
BAD emission line: 2012.7709803030823
BAD emission line: 2032.2510293900418
BAD emission line: 2039.8220177567694
BAD emission line: 2041.2962690324148
BAD emission line: 2041.406378311275
BAD emission line: 2041.8948648728274
GOOD emission line: 1612.4109531253705
BAD emission line: 1635.5890465350606
BAD emission line: 1670.8218459200455
GOOD emission line: 1898.962470491254
GOOD emission line: 1925.1599693251508
GOOD emission line: 1967.2967012029285
BAD emission line: 2007.8954882213652
BAD emission line: 2024.3564326542187
BAD emission line: 2037.5681309720342
BAD emission line: 2042.2080265531802
BAD emission line: 2042.0652712178833
BAD emission line: 2042.607219028383


 50%|█████     | 1/2 [00:16<00:16, 16.85s/it]

GOOD emission line: 1617.7224402742154
BAD emission line: 1649.311859099005
BAD emission line: 1682.6301256781771
BAD emission line: 1729.1696164279863
GOOD emission line: 1908.6937716409163
GOOD emission line: 1931.314485213571
GOOD emission line: 1974.2617652753916
BAD emission line: 2015.7255220665202
BAD emission line: 2029.7443941853576
BAD emission line: 2040.7767256081415
BAD emission line: 2042.2649375537055
GOOD emission line: 1612.8270842019645
BAD emission line: 1633.0028256891894
BAD emission line: 1726.8073638611793
BAD emission line: 1899.2395004300379
GOOD emission line: 1925.2962635126576
GOOD emission line: 1966.8894589434644
BAD emission line: 2006.6096963527796
BAD emission line: 2024.9476508332964
BAD emission line: 2037.0125497495324
BAD emission line: 2041.2813245686696
BAD emission line: 2041.8886561999377


  0%|          | 0/1 [00:00<?, ?it/s]

BAD emission line: 267.85894742285603
GOOD emission line: 74.62512026028934
GOOD emission line: 323.1885631001535
GOOD emission line: 393.374947902989
BAD emission line: 422.92736388408025
GOOD emission line: 445.31885078787144
GOOD emission line: 458.9832052182414
BAD emission line: 471.00103454951017
GOOD emission line: 469.34903500169787
BAD emission line: 653.6605949472322
GOOD emission line: 589.2608562319281
GOOD emission line: 652.8027975167905
GOOD emission line: 692.9389703979443
GOOD emission line: 701.162622688316
GOOD emission line: 756.812659555217
GOOD emission line: 778.341165253083
GOOD emission line: 806.1468753589114
GOOD emission line: 849.041370306964
GOOD emission line: 921.606219561431
GOOD emission line: 964.4201383447147
GOOD emission line: 1056.6394503875865
BAD emission line: 1063.665898827526
BAD emission line: 1101.5965319327352
BAD emission line: 1111.188172208013
BAD emission line: 1155.8849215005864
BAD emission line: 1199.0327270886833
BAD emission line:

100%|██████████| 1/1 [00:59<00:00, 59.17s/it]


In [6]:
# this function is used ONLY if you wish to use x rectification also for y
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.overwrite_rect()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.overwrite_rect()

100%|██████████| 2/2 [00:00<00:00,  5.44it/s]


In [4]:
# calibrate (rectified xs to wls)
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.calib(use_sky=True)

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.calib(use_sky=True)

  0%|          | 0/2 [00:00<?, ?it/s]

BAD PEAK FIT: 1 4026.1914
BAD PEAK FIT: 3 5577.3467
BAD PEAK FIT: 4 5577.3467
BAD PEAK FIT: 20 5577.3467
BAD PEAK FIT: 29 4861.3203
BAD PEAK FIT: 33 4026.1914
BAD PEAK FIT: 34 5577.3467
BAD PEAK FIT: 39 5577.3467
BAD PEAK FIT: 45 5577.3467
BAD PEAK FIT: 51 5577.3467
BAD PEAK FIT: 54 5577.3467
BAD PEAK FIT: 57 5577.3467
BAD PEAK FIT: 58 5577.3467
BAD PEAK FIT: 68 5577.3467
BAD PEAK FIT: 69 5577.3467
BAD PEAK FIT: 78 5577.3467
BAD PEAK FIT: 87 5577.3467
BAD PEAK FIT: 88 5577.3467
BAD PEAK FIT: 91 5577.3467
BAD PEAK FIT: 93 5577.3467
BAD PEAK FIT: 102 5577.3467
BAD PEAK FIT: 112 5577.3467
BAD PEAK FIT: 115 5577.3467
BAD PEAK FIT: 118 5577.3467
BAD PEAK FIT: 119 5577.3467
BAD PEAK FIT: 123 5577.3467
BAD PEAK FIT: 125 5577.3467
BAD PEAK FIT: 130 5577.3467
BAD PEAK FIT: 133 5577.3467
BAD PEAK FIT: 137 5577.3467
BAD PEAK FIT: 139 5895.9321
BAD PEAK FIT: 140 5577.3467
BAD PEAK FIT: 142 5895.9321
BAD PEAK FIT: 143 5577.3467
BAD PEAK FIT: 150 5577.3467
BAD PEAK FIT: 151 5577.3467
BAD PEAK FIT: 1

 50%|█████     | 1/2 [01:32<01:32, 92.52s/it]

BAD PEAK FIT: 11 4026.1914
BAD PEAK FIT: 81 4861.3203
BAD PEAK FIT: 117 4026.1914
BAD PEAK FIT: 122 4026.1914
BAD PEAK FIT: 131 4026.1914
BAD PEAK FIT: 164 4026.1914
BAD PEAK FIT: 196 4861.3203
BAD PEAK FIT: 197 4861.3203
BAD PEAK FIT: 239 5932.8643
BAD PEAK FIT: 242 5932.8643
BAD PEAK FIT: 264 4026.1914
BAD PEAK FIT: 275 5895.9321
BAD PEAK FIT: 300 4026.1914
BAD PEAK FIT: 301 5953.424300000001
BAD PEAK FIT: 307 4026.1914
BAD PEAK FIT: 343 5895.9321
BAD PEAK FIT: 347 4026.1914
BAD PEAK FIT: 365 5932.8643
BAD PEAK FIT: 374 5895.9321
BAD PEAK FIT: 384 4026.1914
BAD PEAK FIT: 401 4026.1914
BAD PEAK FIT: 405 4026.1914
BAD PEAK FIT: 421 4026.1914
BAD PEAK FIT: 422 4026.1914
BAD PEAK FIT: 423 4026.1914
BAD PEAK FIT: 424 4026.1914
BAD PEAK FIT: 424 5889.959
BAD PEAK FIT: 425 4026.1914
BAD PEAK FIT: 426 4026.1914
BAD PEAK FIT: 427 4026.1914
BAD PEAK FIT: 428 4026.1914
BAD PEAK FIT: 429 4026.1914
BAD PEAK FIT: 429 5895.9321
BAD PEAK FIT: 430 4026.1914
BAD PEAK FIT: 431 4026.1914
BAD PEAK FIT: 4

100%|██████████| 2/2 [02:37<00:00, 78.74s/it]


In [6]:
# viz
import matplotlib.pyplot as plt
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

    plt.show()

100%|██████████| 2/2 [00:18<00:00,  9.01s/it]


In [9]:
# viz
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.viz_rect()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.viz_rect()

100%|██████████| 2/2 [00:50<00:00, 25.32s/it]


<h1><strong><span style="color:purple">CALIBRATE</h1>

<span style="color:orange"><strong>calibrate

In [6]:
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.get_spectra(sig_mult,bins,color="b",use_global=True)
    file_for_calib.get_spectra(sig_mult,bins,color="r",use_global=True)

100%|██████████| 2/2 [50:57<00:00, 1528.71s/it]


In [4]:
# intenisty calibrate all spectra, combine blue and red amplifiers
use_flat = False

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.intensity_corr(deg=0,use_flat=use_flat)

  0%|          | 0/2 [00:00<?, ?it/s]

error in intensity fit


100%|██████████| 2/2 [00:54<00:00, 27.47s/it]


In [5]:
# ONLY VIZ

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

100%|██████████| 2/2 [00:35<00:00, 17.58s/it]


<h1><strong><span style="color:purple">STACK</h1>

<span style="color:orange"><strong>stack dithers

In [4]:
# individually create datacubes to be used in dither stack

files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.hex_to_grid()
files_for_stack.spectra_to_datacube()

hexagon grid: 99.000,100.805
pixel grid: 99,101
9999 pixels


overlap percentages: 100%|██████████| 9999/9999 [01:17<00:00, 128.52it/s]


9932 (99.330%) of pixels overlap with hexagon grid
9428 (94.289%) of pixels overlap fully with hexagon grid


graphing pixels: 100%|██████████| 9999/9999 [00:19<00:00, 516.06it/s]


plotting...


graphing pixels: 100%|██████████| 9999/9999 [00:18<00:00, 547.21it/s]


plotting...


 46%|████▋     | 46/99 [01:41<01:56,  2.20s/it]


KeyboardInterrupt: 

In [ ]:
# transform data to WCS using cross-correlation
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.wcs_datacubes()

calculting 2D cross-correlation shift guesses...
select 5 WCS stars in order; see popup window
optimizing WCS star coordinates...
performing WCS transforms...
	0721 used 5 reference stars
	0722 used 5 reference stars
	0723 used 5 reference stars
	0727 used 5 reference stars
	0728 used 5 reference stars
	0729 used 5 reference stars
	0736 used 5 reference stars
	0737 used 5 reference stars
	0738 used 5 reference stars
WCS transformations complete


In [ ]:
# calibrate intensity for all datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.full_intensity_callibration()

In [ ]:
# stack datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.stack_datacubes()

100%|██████████| 99/99 [1:07:33<00:00, 40.94s/it]


final data cubes saved!
